In Shapefiles, information about the coordinate reference system is stored in the .prj -file. If this file is missing, you might be in trouble!. When reading the data into GeoDataFrame with Geopandas crs information is automatically stored into the .crs attribute of the GeoDataFrame.

In [ ]:
import geopandas as gpd

#read the file
fp = r"L2_data/Europe_borders.shp"
data = gpd.read_file(fp)

In [ ]:
#check the coordinate reference system
data.crs

In [ ]:
#verify geometry column to see if CRS of GeoDataFrame is correct
data['geometry'].head() 

#coordinate values of the Polygons indeed look like latitude and longitude values, so everything seems to be in order.

Let’s re-project our data into EPSG 3035 using epsg -parameter:

In [ ]:
#lets make a backup copy of our data
data_wgs84 = data.copy()

#reproject the data ETRS-LAEA projection.
data = data.to_crs(epsg=3055)

In [ ]:
#check the new geometry values
data['geometry'].head()

Plot data with EPSG4326 & EPSG3055

In [ ]:
import matplotlib.pyplot as plt

#make subplots that are next to each other (1x2)
fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2,figsize=(12,12) )

#Plot the data in WGS84 CRS
data_wgs84.plot(ax=ax1, facecolor='gray')

#add title
ax1.set_title("WGS84")

#plot the one with ETRS-LAEA projection
data.plot(ax=ax2, facecolor='blue')

#add title 
ax2.set_title("ETRS Lambert Azimuthal Equal Area Projection")

#set aspect ratio as 1
#ax1.set_aspect(aspect=1)
#ax2.set_aspect(aspect=1)

#remove empty white space around the plot
plt.tight_layout()


In [ ]:
#output filepath of new projection
outfp = "L2_data/Europe_borders_epsg3055.shp"

#save shapefile
data.to_file(outfp)

 pyproj is a Python wrapper around a software called PROJ (maintained by OSGeo community), which is widely used tool for conducting coordinate transformations in various GIS softwares

In [ ]:
### Import CRS class from pyproj
from pyproj import CRS

#proj dictionary
crs_dict = data_wgs84.crs

#pyproj CRS object:
crs_object = CRS(data_wgs84.crs)

#EPSG code (here, the input crs information is a bit vague so we need to lower the confidence threshold
crs_epsg = CRS(data_wgs84.crs).to_epsg(min_confidence=25)

#PROJ string
crs_proj4 = CRS(data_wgs84.crs).to_proj4()

#Well known text (WKT)
crs_wkt = CRS(data_wgs84.crs).to_wkt()

In [ ]:
print("Proj dictionary: \n" , crs_dict)
print("\nCRS object: \n", crs_object)
print("\nEPSG code: \n", crs_epsg)
print("\nPROJ string: \n", crs_proj4)
print("\nWell-Known Text (WKT): \n", crs_wkt)

Pyproj CRS object

In [ ]:
#get current CRS of data
print(data.crs)

In [ ]:
#initialize CRS class for epsg code 3035
crs_object = CRS.from_epsg(3035)
crs_object

parse CRS information individually as follows:

In [ ]:
#get name
print("Name:" , crs_object.name)

#get coordinate system
print("Coordinate system:", crs_object.coordinate_system)

#get Boundas of the area where CRS is used
print("Bounds:", crs_object.area_of_use.bounds)

convert the crs information from one format to another. Quite often it is useful to know the EPSG code of the CRS.

In [ ]:
#retrive CRS information in WKT format
#.prj file of a Shapefile contains the information in this format.

crs_wkt = crs_object.to_wkt()
print(crs_wkt)

extract EPSG code from WKT format

In [ ]:
#Retrieve EPSG code from WKT text
epsg = CRS(crs_wkt).to_epsg()
print(epsg)

Sometimes to_epsg() isn’t able to recognize the EPSG code from the WKT representation. This can happen if the WKT information is missing some details. Luckily, we can easily adjust the minimum level of confidence for matching the CRS info and the EPSG code. We can do this by adjusting a parameter min_confidence when calling the function. By default, the confidence level is 70 %, but it is also possible to set a lower confidence threshold.

Save data in WKT format as the CRS of GEODATAFRAME. WKT is preferred output format when storing CRS info as text

In [ ]:
#re-define the CRS of the input GeoDataFrame -> convert to WKT
data.crs = CRS.from_epsg(3035).to_wkt()
print(data.crs)

In [ ]:
#output filepath
outfp = "L2_data/Europe_borders_epsg3035.shp"

#save to disk
data.to_file(outfp)

Global Map Projections

In [ ]:
#read in data
fp = "L2_data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp"
admin = gpd.read_file(fp)

In [ ]:
#check input crs
admin.crs

In [ ]:
#set figure size
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
#plot in original CRS
admin.plot()
plt.title("WGS84")

In [ ]:
#Define projections as web mercator, 3785
web_mercator = CRS.from_epsg(3785)

#re-project and plot
admin.to_crs(web_mercator).plot()

#Remove x and y axis
plt.axis('off')
plt.title("Web mercator")

In [ ]:
#Define projection Eckert IV from https://spatialreference.org/ref/esri/54012/
eckert_IV = CRS.from_proj4("+proj=eck4 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

#Re-project and plot
admin.to_crs(eckert_IV).plot()

#Remove x and y axis
plt.axis('off')
plt.title("Eckert IV")

In [ ]:
#Define an orthographic projection, centered in Finland! from: http://www.statsmapsnpix.com/2019/09/globe-projections-and-insets-in-qgis.html
ortho = CRS.from_proj4("+proj=ortho +lat_0=60.00 +lon_0=23.0000 +x_0=0 +y_0=0 +a=6370997 +b=6370997 +units=m +no_defs")

#re-project and plot
admin.to_crs(ortho).plot()

#remove x and y axis
plt.axis('off')
plt.title("Orthographic")